Importations

In [131]:
import mediapipe as mp 
import numpy as np
import cv2
import joblib
import json
from  collections import deque
from analyse_pose import extract_ratio
from annotate import extract_angles,calculate_angle
import os


Initialisations :

In [132]:
mp_pose=mp.solutions.pose
pose=mp_pose.Pose(min_detection_confidence=0.6,min_tracking_confidence=0.5)
#si la tracking_confidence< min_tracking_confidence :la modele reinitialise 
#la dection de pose 
mp_drawing = mp.solutions.drawing_utils
# Chargement des modèles
model = joblib.load(r'C:\Users\lanouar\sources\Activity_recognition\shoulder_press_counting\model2_shoulder_press.pkl')
label_encoder = joblib.load(r'C:\Users\lanouar\sources\Activity_recognition\shoulder_press_counting/label_encoder_shoulder_press.pkl')
prev_landmarks = {}
window_size=5
Landmark = mp_pose.PoseLandmark
# Chargement des seuils
with open(r'C:\Users\lanouar\sources\Activity_recognition\shoulder_press_counting/thresholds2.json', 'r') as f:
    thresholds = json.load(f)

fonctions for smooth pose and   angle calculation :

In [133]:
def smooth_point(prev_point,new_point,alpha=0.5):
    return alpha*prev_point + new_point*(1-alpha)
def smooth_angles(new_angles,angle_history):
    smoothed={}
    for name,value in new_angles.items():
        if name not in angle_history:
            angle_history[name]=deque(maxlen=window_size)
        angle_history[name].append(value)
        smoothed[name]=np.mean(angle_history[name])
    return smoothed
def calculate_hip_shoulder_elbow_angle(lm):
   angle = calculate_angle(
        [lm[Landmark.RIGHT_HIP].x, lm[Landmark.RIGHT_HIP].y],
        [lm[Landmark.RIGHT_SHOULDER].x, lm[Landmark.RIGHT_SHOULDER].y],
        [lm[Landmark.RIGHT_ELBOW].x, lm[Landmark.RIGHT_ELBOW].y]
    )
   return angle
        


In [134]:
#fonction pour tester  si l'angle calculer est correct
def in_thresholds(angle, min_val, max_val):
    return ((min_val-10) <= angle <= (max_val+15))
#fonction pour detecter la phase encours:
def predict_phase(angles):
   features = np.array([[angles['right_shoulder'], angles['left_shoulder']]])
#1 car on veut un seul ligne: un seul frame a predicter 
#-1 calcul automatique de nombre des colonnes qui doit etre 2 
   phase_encoded=model.predict(features)[0]
   return label_encoder.inverse_transform([phase_encoded])[0]


In [135]:
def test_video(video_path):
 current_phase = None
 angle_history={}
 counter = 0
 feedback = ""
 score = 0
 phase = None
 y_offset = 150
 cap=cv2.VideoCapture(video_path)
 if not cap.isOpened():
     print('erreur d''ouverture')
 while cap.isOpened():
  ret,frame=cap.read()
  if not ret:
       break
  badratio = {'elbow': '', 'wrist': ''}
  shoulder_angle=np.nan
  frame = cv2.resize(frame, (1200, 700))
 #transformer l'image de RGB(par defaut retourner OpenCV ) to BGR pour mediapipe 
  image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  results = pose.process(image_rgb)
  if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        raw_landmarks = results.pose_landmarks.landmark
        smoothed_landmarks = []

        for i, lm in enumerate(raw_landmarks):
                        new_point = np.array([lm.x, lm.y, lm.z])
                        if i in prev_landmarks:
                            prev_point = prev_landmarks[i]
                            smoothed = smooth_point(prev_point, new_point)
                        else:
                            smoothed = new_point
                        prev_landmarks[i] = smoothed

                        class SmoothedLandmark:
                            def __init__(self, x, y, z, visibility):
                                self.x = x
                                self.y = y
                                self.z = z
                                self.visibility = visibility

                        smoothed_landmarks.append(SmoothedLandmark(
                            smoothed[0], smoothed[1], smoothed[2], lm.visibility))

                    # Utiliser les points lissés
        angles = extract_angles(smoothed_landmarks)
        ratio=extract_ratio(smoothed_landmarks)
        smoothed_angles = smooth_angles(angles,angle_history)
        shoulder_angle = calculate_hip_shoulder_elbow_angle(smoothed_landmarks)
        phase = predict_phase(smoothed_angles)
        bad_angles = {}

        if phase != 'milieu':
                for angle_name, angle_value in angles.items():
                    if not np.isnan(angle_value):
                        min_val, max_val = thresholds[phase][angle_name][0],thresholds[phase][angle_name][1]
                        if not in_thresholds(angle_value, min_val, max_val):
                            bad_angles[angle_name] = angle_value

                for angle_name, angle_value in bad_angles.items():
                    min_val, max_val = thresholds[phase][angle_name][0],thresholds[phase][angle_name][1]
                    text = f"{angle_name}: {angle_value:.1f} (Out of [{min_val}-{max_val}])"
                    print(text)
                    cv2.putText(frame, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                    y_offset += 25

                min_w, max_w = thresholds[phase]['wrist'][0],thresholds[phase]['wrist'][1]
                min_e, max_e = thresholds[phase]['elbow'][0],thresholds[phase]['elbow'][1]

                if not np.isnan(ratio['wrist']) and not in_thresholds(ratio['wrist'], min_w, max_w):
                    badratio['wrist'] = "les mains sont rapprochées" if ratio['wrist'] < min_w else "écartez un peu les mains"
                if not np.isnan(ratio['elbow']) and not in_thresholds(ratio['elbow'], min_e, max_e):
                    badratio['elbow'] = "les coudes sont rapprochés" if ratio['elbow'] < min_e else "écartez un peu les coudes"

        # Analyse des phases
        if phase == "haut":
            if not bad_angles :
                if not np.isnan(shoulder_angle)and shoulder_angle>150 :
                 feedback = "bon posture"
                elif  current_phase != "haut":
                    feedback='your position is incorrect'
                current_phase = "haut"
            else:
                feedback = "Mauvaise posture (haut)"
        elif phase == "bas" :
            if not bad_angles and current_phase == 'haut':
                 feedback = "shoulder press valide !"
                 counter += 1
                 current_phase='bas'
                 score += 5 
        else:
            feedback = ""

        # UI
        h, w = frame.shape[:2]
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (w, 140), (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.6, frame, 0.4, 0)

        font = cv2.FONT_HERSHEY_SIMPLEX
        white = (255, 255, 255)
        green = (0, 255, 0)
        red = (0, 0, 255)
        feedback_color = red if "Mauvaise" in feedback else green

        cv2.putText(frame, f"Phase : {phase}", (10, 25), font, 0.7, white, 2)
        cv2.putText(frame, f"Repetitions : {counter}", (10, 55), font, 0.7, green, 2)
        cv2.putText(frame, f"Score : {score}", (10, 85), font, 0.7, green, 2)
        cv2.putText(frame, f"Feedback : {feedback}", (10, 115), font, 0.6, feedback_color, 2)

        if badratio['elbow']:
            cv2.putText(frame, f"elbow : {badratio['elbow']}", (10, 155), font, 0.6, red, 1)
        if badratio['wrist']:
            cv2.putText(frame, f"wrist : {badratio['wrist']}", (10, 175), font, 0.6, red, 1)

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow("Phase shoulder_press - Prédiction", frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

 cap.release()
 cv2.destroyAllWindows()



In [137]:
video_path = r'C:\Users\lanouar\sources\Activity_recognition\dataset\shoulder press\shoulder press_5.mp4'
print('video-start')
test_video(video_path)

video-start
